In [1]:
import pandas as pd
import numpy as np
# Preprocessing
import preprocessor as p
import string
import contractions
from collections import defaultdict
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk import pos_tag
from nltk.corpus import stopwords
# Model
from sklearn import model_selection, svm
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
# Pipeline
from sklearn.pipeline import make_pipeline
from joblib import dump

In [2]:
# Download requires corpus
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cgab\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cgab\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\cgab\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\cgab\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cgab\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# Read data
df = pd.read_csv("../res/dataset-final.csv")

train_df = pd.read_csv("../res/train_final.csv")
val_df = pd.read_csv("../res/val_final.csv")
test_df = pd.read_csv("../res/test_final.csv")
train_df.head()

,text,HATE,NOT,OFFN,PRFN,Race,Religion,Gender,Other,None
0,a vote for max is a vote for the white christi...,0,1,0,0,0,0,0,0,1
1,what a muslim does in the privacy of his own g...,1,0,0,0,1,1,0,0,0
2,it is in some circles but in others it the sam...,1,0,0,1,1,0,0,0,0
3,president trumps opinion get the son of the bi...,0,1,1,1,1,0,0,1,1
4,id2020 is also jabbing refugees with its micro...,0,1,1,0,0,0,0,1,1


In [4]:
# df = df.iloc[:500]

In [5]:
# Organize targets
# Get target columns
headers = train_df.columns[1:]
# Create separate df for targets
y_train = train_df[headers]
y_val = val_df[headers]
y_test = test_df[headers]


In [6]:
t1a2_headers = headers[:4]
print("subtask 1 and 2 headers:")
print(t1a2_headers)
t3_headers = headers[4:]
print("subtask 3 headers:")
print(t3_headers)

subtask 1 and 2 headers:
Index(['HATE', 'NOT', 'OFFN', 'PRFN'], dtype='object')
subtask 3 headers:
Index(['Race', 'Religion', 'Gender', 'Other', 'None'], dtype='object')


In [7]:
# Tokenize text column
# train_df['text']= [str(word_tokenize(entry)) for entry in train_df['text']]
# val_df['text']= [str(word_tokenize(entry)) for entry in val_df['text']]
# test_df['text']= [str(word_tokenize(entry)) for entry in test_df['text']]
# train_df['text'].head()

Reference for multilabel preprocessing: https://www.section.io/engineering-education/multi-label-classification-with-scikit-multilearn/

In [8]:
asdgasdasd

NameError: name 'asdgasdasd' is not defined

<H1>Creation of Model </H1>

In [13]:
# pip install scikit-multilearn
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.multiclass import OneVsRestClassifier
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset

<h4>BinaryRevelance</h4>

In [ ]:
br_classifier = make_pipeline(
    TfidfVectorizer(),
    BinaryRelevance(svm.SVC(kernel='linear', probability=True))
)
br_classifier = br_classifier.fit(X=train_x, y=train_y)
br_classifier

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('binaryrelevance',
                 BinaryRelevance(classifier=SVC(kernel='linear',
                                                probability=True),
                                 require_dense=[True, True]))])

In [ ]:
predictions_br = br_classifier.predict(test_x)
print("SVM Accuracy Score -> ",accuracy_score(predictions_br, test_y))

SVM Accuracy Score ->  0.40066666666666667


<h4>One vs Rest</h4>

In [14]:
ovr_classifier = make_pipeline(
    TfidfVectorizer(),
    OneVsRestClassifier(svm.SVC(kernel='linear', probability=True))
)

In [15]:
ovr_classifier_t1a2 = ovr_classifier.fit(X=train_df["text"][:500], y=train_df[t1a2_headers][:500])
ovr_classifier_t1a2

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('onevsrestclassifier',
                 OneVsRestClassifier(estimator=SVC(kernel='linear',
                                                   probability=True)))])

In [ ]:
filename = "ovr_classifier_t1a2.joblib.z"
path = "../out/subtask3/" + filename
dump(ovr_classifier_t1a2, path)

['../out/subtask3/ovr_classifier_t1a2.joblib.z']

In [ ]:
ovr_classifier_t3 = ovr_classifier.fit(X=train_df["text"], y=train_df[t3_headers])
ovr_classifier_t3

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('onevsrestclassifier',
                 OneVsRestClassifier(estimator=SVC(kernel='linear',
                                                   probability=True)))])

In [ ]:
filename = "ovr_classifier_t3.joblib.z"
path = "../out/subtask3/" + filename
dump(ovr_classifier_t3, path)

['../out/subtask3/ovr_classifier_t3.joblib.z']

In [ ]:
predictions_t1a2 = ovr_classifier_t1a2.predict(test_df["text"])
print("SVM Accuracy Score -> ",accuracy_score(predictions_t1a2, test_df[t1a2_headers]))

ValueError: inconsistent shapes

<h4>Classifer Chain</h4>

In [ ]:
cf_classifier = make_pipeline(
    TfidfVectorizer(),
    ClassifierChain(svm.SVC(kernel='linear', probability=True))
)
cf_classifier = cf_classifier.fit(X=train_x, y=train_y)
cf_classifier

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('classifierchain',
                 ClassifierChain(classifier=SVC(kernel='linear',
                                                probability=True),
                                 require_dense=[True, True]))])

In [ ]:
predictions_cf = cf_classifier.predict(test_x)
print("SVM Accuracy Score -> ",accuracy_score(predictions_cf, test_y))

SVM Accuracy Score ->  0.4013333333333333


<h4>Label Powerset</h4> 

In [ ]:
lp_classifier = make_pipeline(
    TfidfVectorizer(),
    LabelPowerset(svm.SVC(kernel='linear', probability=True))
)
lp_classifier = lp_classifier.fit(X=train_x, y=train_y)
lp_classifier

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('labelpowerset',
                 LabelPowerset(classifier=SVC(kernel='linear',
                                              probability=True),
                               require_dense=[True, True]))])

In [ ]:
predictions_lp = lp_classifier.predict(test_x)
print("SVM Accuracy Score -> ",accuracy_score(predictions_lp, test_y))

SVM Accuracy Score ->  0.38466666666666666


In [ ]:
classifer_names = ["BinaryRelevance", "OneVSRest", "ClassifierChain", "LabelPowerset"]
classifiers = [br_classifier, ovr_classifier, cf_classifier, lp_classifier]

<h2>Make a Single prediction</h2>

In [17]:
# Create preprocessing function
def preprocess(text):
    # twitter-presprocessor
    text = p.clean(text)
    # lower case
    text = text.lower()
    # Expand Contractions
    expanded = []
    for word in text.split():
        expanded.append(contractions.fix(word))
    text = ' '.join(expanded)
    # Remove punctuation marks
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenization
    # text = word_tokenize(text)
    
    # lemmanized = preprocess_part2(text)

    # return str(text)
    return text

def preprocess_part2(text):
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(text):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    return str(Final_words)


In [20]:
text1 = "Girl bitches should just kill themselves if they don't know how to cook"
text2 = "gay people are mentally ill"
text3 = "Islam people are all terrorists"
text4 = "Asians should just go back to their country, all they do is take our jobs"
text5 = "If you want to make the world a better place, look at yourself and make a change"
sample_texts = [text1, text2, text3, text4, text5]

for text in sample_texts:
    arr = [text]
    classes = ovr_classifier_t1a2.classes_
    proba = ovr_classifier_t1a2.predict_proba(arr)[0]
    pred = ovr_classifier_t1a2.predict(arr)[0]

    headers = t1a2_headers

    print(f'INPUT: {text}')
    for i,e in enumerate(classes):
        print(f'  {headers[i]}: {proba[i]*100}')
    print(f'PREDICTION: {pred}\n')

INPUT: Girl bitches should just kill themselves if they don't know how to cook
  HATE: 26.493388689622805
  NOT: 81.3603040639828
  OFFN: 50.61964021029951
  PRFN: 95.8381764708028
PREDICTION: [0 1 0 1]

INPUT: gay people are mentally ill
  HATE: 26.28186645277566
  NOT: 46.82698475284533
  OFFN: 49.106084938958396
  PRFN: 79.83873674663634
PREDICTION: [0 0 0 1]

INPUT: Islam people are all terrorists
  HATE: 54.03054103145939
  NOT: 24.0550798618161
  OFFN: 56.29807360208757
  PRFN: 74.26919916727036
PREDICTION: [1 0 1 1]

INPUT: Asians should just go back to their country, all they do is take our jobs
  HATE: 70.25943681431573
  NOT: 25.72041841591219
  OFFN: 64.76602469990706
  PRFN: 94.31976531542702
PREDICTION: [1 0 1 1]

INPUT: If you want to make the world a better place, look at yourself and make a change
  HATE: 19.338091037855243
  NOT: 48.11704586310737
  OFFN: 41.72321055884182
  PRFN: 58.89531050221133
PREDICTION: [0 0 0 1]



In [ ]:
for i,classifier in enumerate(classifiers):
    print(f"------------ {classifer_names[i]} ------------\n")
    for text in sample_texts:
        arr = [preprocess(text)]
        classes = classifier.classes_
        proba = classifier.predict_proba(arr)[0]
        pred = classifier.predict(arr)[0]

        headers = y.columns

        print(f'INPUT: {text}')
        for i,e in enumerate(classes):
            print(f'  {headers[i]}: {proba[i]*100}')
        print(f'PREDICTION: {pred}\n')

------------ BinaryRelevance ------------



AttributeError: 'BinaryRelevance' object has no attribute 'classes_'

<h2>Export Classifier Models</h2>

In [ ]:
# Export model as joblib file
for i,classifier in enumerate(classifiers):
    filename = classifer_names[i] + ".joblib.z"
    path = "../out/subtask3/" + filename
    dump(classifier, path)

<H2>Load Models</H2>

In [ ]:
from joblib import load 

In [ ]:
text1 = "Girl bitches should just kill themselves if they don't know how to cook"
text2 = "gay people are mentally ill"
text3 = "Islam people are all terrorists"
text4 = "Asians should just go back to their country, all they do is take our jobs"
text5 = "If you want to make the world a better place, look at yourself and make a change"
text6 = "The mentally ill should seek help no cap"
sample_texts = [text1, text2, text3, text4, text5, text6]

In [ ]:
# Initialize
path = "../out/subtask3/"
extension = ".joblib.z"
br_classifier = load(path + "BinaryRelevance" + extension)
ovr_classifier = load(path + "OneVSRest" + extension)
cf_classifier = load(path + "ClassifierChain" + extension)
lp_classifier = load(path + "LabelPowerset" + extension)

In [ ]:
classifer_names = ["BinaryRelevance", "ClassifierChain", "LabelPowerset"]
classifiers = [br_classifier, cf_classifier, lp_classifier]
headers = y.columns

In [ ]:
for i,classifier in enumerate(classifiers):
    print(f"------------ {classifer_names[i]} ------------\n")
    for text in sample_texts:
        arr = [preprocess(text)]
        proba = classifier.predict_proba(arr).toarray()[0]
        pred = classifier.predict(arr).toarray()[0]

        print(f'INPUT: {text}')
        for i,e in enumerate(proba):
            print(f'  {headers[i]}: {e*100}')
        print(f'PREDICTION: {pred}\n')

------------ BinaryRelevance ------------

INPUT: Girl bitches should just kill themselves if they don't know how to cook
  Homophobe: 3.2266965875375706
  NotHate: 95.10907093835792
  OtherHate: 15.938275183210504
  Racist: 26.94560942783787
  Religion: 1.5721904256209234
  Sexist: 14.230099565607837
PREDICTION: [0 1 0 0 0 0]

INPUT: gay people are mentally ill
  Homophobe: 47.723661884801714
  NotHate: 90.20893668375912
  OtherHate: 53.00006168680037
  Racist: 25.009825716168443
  Religion: 1.4332809260186665
  Sexist: 10.687020317583498
PREDICTION: [1 1 1 0 0 0]

INPUT: Islam people are all terrorists
  Homophobe: 6.562874384697421
  NotHate: 94.34908851227136
  OtherHate: 11.297616861267496
  Racist: 30.231812673323017
  Religion: 50.0
  Sexist: 7.717969503386206
PREDICTION: [0 1 0 0 0 0]

INPUT: Asians should just go back to their country, all they do is take our jobs
  Homophobe: 2.236149474555906
  NotHate: 93.1774648952431
  OtherHate: 12.799872117093495
  Racist: 44.5940430756